# Hierarchical Clustering on Target Sentences
to help deriving a taxonomy for intents, hierarchical clustering is being leveraged.

#### Functions

In [1]:
def inspect_topic(topic_nr):
    print('-----------')
    top_words = topic_model.get_topic(topic_nr)
    print("Topic: "+str(topic_nr)+"\n\nTop Words:")
    for word in top_words:
        print(word[0], end=", ")
    print('\n\nRepresentative Target Sentences: ')
    for doc in topic_model.get_topic_info().iloc[topic_nr+1]['Representative_Docs']:
        print(doc)

def create_custom_label(custom_label, topic_id):
    custom_labels = {
        topic_id: str(topic_id)+"_"+custom_label,
    }
    topic_model.set_topic_labels(custom_labels)

# Function to retrieve custom label from topic model
def get_label_from_topic_nr(topic_nr):
    for label in topic_model.custom_labels_:
        if label.startswith(str(topic_nr)):
            label = label
            label = re.sub(r"^\d+_", "", label)
            return label
    raise Exception('Provided Topic Number has no label assigned!')

# Function to inspect a topic by its number and retrieve its documents
'''
def inspect_topic_and_its_docs(topic_nr,n_docs=50):
    topic_docs = list(set([doc for doc, topic in zip(docs, topic_model.topics_) if topic == topic_nr]))
    print("Label: " + get_label_from_topic_nr(topic_nr)+"\n")
    print("Number of sentences for label: "+str(len(topic_docs)))
    print('\nSentences:\n')
    if len(topic_docs) < n_docs:
        n_docs = len(topic_docs)
    for i in range(n_docs):
        print(topic_docs[i])
    return topic_docs
'''

def inspect_topic_docs(topic_id, n_docs=20):
    documents = [docs[i] for i, topic in enumerate(topic_model.topics_) if topic == topic_id]
    print('Number of Documents: '+str(len(documents))+'\n')
    if len(documents) < n_docs:
        n_docs = len(documents)
    for i in range(n_docs):
        print(documents[i])
    return documents

# given a list of strings, returns a list of documents that contain at least one of the strings in the given list
def filter_topic_documents(topic_docs, white_list, black_list=[] ):
    sentences_to_keep = []
    sentences_to_discard = []
    for doc in topic_docs:
        if any(substring in doc for substring in white_list):
            sentences_to_keep.append(doc)
    sentences_to_keep = [
    item for item in sentences_to_keep
        if not any(substring in item for substring in black_list)
    ]
    sentences_to_keep = list(set(sentences_to_keep)) # remove duplicates
    sentences_to_discard = list(set(topic_docs) - set(sentences_to_keep))
    print('Amount of sentences: '+str(len(sentences_to_keep)))
    return sentences_to_keep, sentences_to_discard

# Function to facilitate the concatenation of new DFs
def concat_to_df(new_df,filepath, overwrite=False, ):
    try:
        df = pd.read_csv(filepath, sep='\t')
    except FileNotFoundError:
        df = pd.DataFrame(columns=["text", "label"])
    if new_df.iloc[0]['label'] not in df['label'].values: # Make sure that DF does not already contain entry for label
        new_df = new_df.reset_index(drop=True)
        df = df.reset_index(drop=True)
        df = pd.concat([df, new_df], ignore_index=True)
        df = df.reset_index(drop=True)
        df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
        df.to_csv(filepath, sep='\t')
        return df
    else:
        if overwrite:
            df = df[~df['label'].str.contains(new_df.iloc[0]['label'])]
            df = pd.concat([df, new_df], ignore_index=True)
            df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
            df.to_csv("../../data/labeled/send.tsv", sep='\t', index=False)
            return df
        else:
            print("Label '"+new_df.iloc[0]['label']+"' already in the final DF, so it's being skipped.")


####

In [2]:
from bertopic import BERTopic
from scipy.cluster import hierarchy as sch
import pandas as pd
import logging
from sklearn.metrics.pairwise import cosine_distances
from tqdm import tqdm
import numpy as np
from umap import UMAP
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
import re

In [4]:
df = pd.read_parquet('../../../data/processed/intents/do.parquet', engine='fastparquet')

In [5]:
df['target'] = df['target'].str.replace(r'\r\n', ' ', regex=True)
docs = list(set(df['target'].tolist()))

In [6]:
# Amount of unique target sentences
len(docs)

16215

## Training the Model

In [7]:
# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("BERTopic")
logger.setLevel(logging.INFO)

In [8]:
# Initialize UMAP with a fixed random state
random_state = 1 # Other seeds sometimes caused negative values in the distanace matrix

# Set global random seeds
np.random.seed(random_state)

embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
umap_model = UMAP(n_neighbors=15, n_components=5, random_state=random_state)
hdbscan_model = HDBSCAN(min_cluster_size=10, prediction_data=True)

topic_model = BERTopic(verbose=True, umap_model=umap_model, hdbscan_model=hdbscan_model, embedding_model=embedding_model)
topics, probs = topic_model.fit_transform(docs)

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
2024-11-06 20:47:35,588 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/507 [00:00<?, ?it/s]

2024-11-06 20:48:03,823 - BERTopic - Embedding - Completed ✓
2024-11-06 20:48:03,823 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
2024-11-06 20:48:20,210 - BERTopic - Dimensionality - Completed ✓
2024-11-06 20:48:20,211 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-11-06 20:48:20,694 - BERTopic - Cluster - Completed ✓
2024-11-06 20:48:20,697 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-11-06 20:48:20,906 - BERTopic - Representation - Completed ✓


In [9]:
hierarchical_topics = topic_model.hierarchical_topics(docs)

100%|██████████| 154/154 [00:00<00:00, 634.07it/s]


In [11]:
topic_model.save("../../../data/bertopic_models/intents/04_do/do_unprocessed")

2024-11-06 20:49:23,318 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


## Inspecting clustering results

In [7]:
#topic_model = BERTopic.load("../../data/bertopic_models/send")

In [53]:
topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)

## Merging Topics

In [52]:
topic_id = 7
inspect_topic(topic_id)

-----------
Topic: 7

Top Words:
japanese, translation, english, localization, german, language, documents, characters, kanji, i18n, 

Representative Target Sentences: 
All ,    I know we have done Japanese Kanji on a Japanese phone , but what about Chinese ?   Also , are there any devices in [LOCATION] that display Kanji characters ?   Does [PERSON] support Chinese on RIM ?
I would like to show that we can do part of their site in i - mode Japanese .
So you can use LANGUAGE_COUNTRY="en_US " for every charset ( This will show all UI strings in english).Because we do not have translation available for other languages except for japanese which you have done earlier .


In [14]:
#topic_model.custom_labels_

In [49]:
topic_model.merge_topics(docs, [-1,topic_id])

In [732]:
create_custom_label("it_problems", topic_id=topic_id)

In [39]:
topic_docs = inspect_topic_docs(topic_id,n_docs=20)

Number of Documents: 122

do processing and return the result to the jsp page ( result show in   parent   > > window ) .
On the other hand , if [LOCATION] is a JavaScript object , and you want the   invoked JSP to have access to the object itself , I do n't think you can do   that .
Personally I have logic being mixed in with presentation , as such I 'd limit   the JSP code as much as possible , the way I 'd do this is by writing all the   logic as Beans which either then access the database directly or through   EJB 's .
At the moment I do web applications ( Java / Servlets / XML )    > >
How can i do it in JSP .
Oh , wait , you want JSP to do it - which means   the client does n't have anything to do with it at * all * except to issue a   request to the JSP container .
Cc : piyush goel   Sent : Friday , June 11 , 1999 7:36 AM   Subject : client interface     rajeev    i havea rudimentary jni thing working , but in order to integrate   i need to do some more things .
How can i do it i

In [427]:
len(topic_model.get_topics())

59

In [485]:
topic_model.save("../../../data/bertopic_models/intents/02_call/call_processed")

2024-11-06 15:37:29,646 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


In [707]:
topic_model.visualize_hierarchy(custom_labels=False)

In [1121]:
topic_model.save("../../../data/bertopic_models/intents/02_call/call_final")

2024-11-06 17:34:41,016 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


### Create Dataset for intent 'help'

####  Labels

Created custom labels for topics

| Label                             | Description                                       | Extraction Strings | Context needed |
|-----------------------------------|---------------------------------------------------|--------------------|----------------|
| '0_offer_help',
| '1_request_help',
| '2_jsp_java_code',
| '3_refer_help',
| '4_person_help_out_you',
| '5_it_problems',

#### Inspecting Topics

In [1672]:
'''
for  label in topic_model.custom_labels_:
   print(label)'''

'\nfor  label in topic_model.custom_labels_:\n   print(label)'

In [3]:
topic_model = BERTopic.load("../../data/bertopic_models/processed/send_final")

In [758]:
topic_id = 5
topic_docs = inspect_topic_docs(topic_id=topic_id, n_docs=20)

Number of Documents: 106

I also think that we will need help from [PERSON] on fixing some of the bugs which are filed by [PERSON] and hard to reproduce at [LOCATION] [LOCATION] .
Micro$oft software is having problems similar to this and there have been patches made available for this problem for a couple weeks .
Ravi ,   this is an interesting scenario , I think we have some problems with that , that needs to be fixed .
Simple errors like this should n't require [PERSON] 's help to resolve , but she is getting pulled in to them .
Hi Shailesh ,         Please try this patch for the project having problem .
Clone of ' [PERSON] ' virus infects the Internet    Promising help in the mating game , a new Trojan virus similar to the " [PERSON] " bug is proliferating quickly across the Internet , computer security specialists cautioned this week .    - FULL STORY -    Sci - Tech index 	  -Next category- 	  - Back to top - 	      _ _ _ _ _
We did n't have the second problem when we tested app l

In [759]:
# Based on the inspection of the sentences
white_list = [
    ""
]
black_list = [

]

In [760]:
sents_to_keep, sents_to_discard = filter_topic_documents(topic_docs=topic_docs, white_list=white_list, black_list=black_list)
sents_to_keep

Amount of sentences: 106


["My 2 cents :    - Old open bugs : Bugs can not be closed using [PERSON] 's help                            without seeing whether those bugs are relevant or not .",
 'have you had any problem with updating this spread sheet .',
 'But that has not helped me in pointing out the problem on the colocation server .',
 'Please take a look at this and see if you can reproduce this using the latest build .',
 "We did n't have the second problem when we tested app locally , but when we put it on COLO , it happened .",
 '[PERSON]    Can you help get some more information on this bug ?    Thanks   Srik',
 'Also , can you have [PERSON] take a look again at the 8 purge failures   in OSA .',
 "We did n't have the second problem when we tested app locally , but when we   > put it on COLO , it happened .",
 "[PERSON] ,    We may need Sreenivas ' help to reproduce bugs during weekend .",
 'Let us fix this then , this seems like a totally internal issue which will   help us all .',
 'If you need some 

In [761]:
sents_to_discard

[]

In [762]:
fp = "../../../data/labeled/help.tsv"
df = pd.DataFrame({"text": sents_to_keep, "label": get_label_from_topic_nr(topic_id)})
concat_to_df(df, filepath=fp, overwrite=False)

,text,label
0,Please let me know if you have any questions .,offer_help
1,"( [PERSON] ) If you have questions , please...",offer_help
2,> > Please let me know if you have any quest...,offer_help
3,"If you have any questions , please do not he...",offer_help
4,Please let me know if this helps and if you ha...,offer_help
...,...,...
987,[PERSON] is having some problems with the LC20...,it_problems
988,So I 'll try my best to fix the problems on th...,it_problems
989,[PERSON] does this have the changes [PERSON...,it_problems
990,This worked until a couple of weeks ago . H...,it_problems


In [1223]:
df.head()

,text,label
0,Once you have had a chance to review and if yo...,call_because_of_changes
1,All the changes are in TradingHistoryForm.aml ...,call_because_of_changes
2,Please give me a call if you have any question...,call_because_of_changes
3,If you 'd like to have the order changed to Di...,call_because_of_changes
4,"blair , please review and call me asap thank...",call_because_of_changes
